In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import malaya

tokenize = malaya.tokenizer.Tokenizer().tokenize
#text = ' '.join(tokenize(text))

/home/ubuntu/.local/lib/python3.8/site-packages/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3361
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/ubuntu/.local/lib/python3.8/site-packages/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3879
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('mesolitica/t5-small-standard-bahasa-cased')

In [4]:
from glob import glob

checkpoints = sorted(glob('finetune-t5-small-standard-bahasa-cased-ner-new/checkpoint-*'))
checkpoints

['finetune-t5-small-standard-bahasa-cased-ner-new/checkpoint-510000',
 'finetune-t5-small-standard-bahasa-cased-ner-new/checkpoint-520000',
 'finetune-t5-small-standard-bahasa-cased-ner-new/checkpoint-530000']

In [5]:
model = T5ForConditionalGeneration.from_pretrained(checkpoints[-1])

In [6]:
from unidecode import unidecode
import re

# minimum cleaning, just simply to remove newlines.
def cleaning(string):
    string = string.replace('\n', ' ')
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

x = """
Bekas Ahli Parlimen Jempol empat penggal Khalid Yunus akan membuat penampilan semula pada pilihan raya umum ke-15 (PRU15).

Khalid, 79, yang juga timbalan presiden Parti Bumiputera Perkasa Malaysia (Putra) akan bertanding atas tiket Pejuang menentang Shamshulkahar Mohd Deli daripada Barisan Nasional (BN), calon Perikatan Nasional (PN) Noraffendy Salleh dan calon Pakatan Harapan (PH) Norwani Ahmat.
"""

text = cleaning(x)
text = ' '.join(tokenize(text))
tags = ['nama seseorang', 'organisasi', 'kopi', 'parti politik', 'masa', 'food and beverages',
       'age', 'politician', 'ahli politik', 'orang politik', 'parti politik malaysia']
input_ids = []
for l in tags:
    s = f'Ekstrak {l} dari teks: {text}'
    print(s)
    # s = f'teks: {text} entiti: {t}'
    input_ids.append({'input_ids': tokenizer.encode(s, return_tensors='pt')[0]})

padded = tokenizer.pad(input_ids, padding='longest')
outputs = model.generate(**padded, max_length = 256)
b = tokenizer.batch_decode(outputs, skip_special_tokens=True)
for no, l in enumerate(tags):
    print(l, b[no].split(' dan '))

Ekstrak nama seseorang dari teks: Bekas Ahli Parlimen Jempol empat penggal Khalid Yunus akan membuat penampilan semula pada pilihan raya umum ke-15 ( PRU15 ) . Khalid , 79 , yang juga timbalan presiden Parti Bumiputera Perkasa Malaysia ( Putra ) akan bertanding atas tiket Pejuang menentang Shamshulkahar Mohd Deli daripada Barisan Nasional ( BN ) , calon Perikatan Nasional ( PN ) Noraffendy Salleh dan calon Pakatan Harapan ( PH ) Norwani Ahmat .
Ekstrak organisasi dari teks: Bekas Ahli Parlimen Jempol empat penggal Khalid Yunus akan membuat penampilan semula pada pilihan raya umum ke-15 ( PRU15 ) . Khalid , 79 , yang juga timbalan presiden Parti Bumiputera Perkasa Malaysia ( Putra ) akan bertanding atas tiket Pejuang menentang Shamshulkahar Mohd Deli daripada Barisan Nasional ( BN ) , calon Perikatan Nasional ( PN ) Noraffendy Salleh dan calon Pakatan Harapan ( PH ) Norwani Ahmat .
Ekstrak kopi dari teks: Bekas Ahli Parlimen Jempol empat penggal Khalid Yunus akan membuat penampilan semu

In [7]:
text = 'olly saya nak secawan kopi dengan satu krim dan tiga gula, dan saya sekarang berada di penang airport'
text = ' '.join(tokenize(text))
tags = ['person', 'organisasi', 'kopi', 'nama parti politik', 'lokasi', 'syarikat', 'masa',
       'quantity', 'kuantiti', 'coffee', 'airport', 'politician']
input_ids = []
for l in tags:
    s = f'Ekstrak {l} dari teks: {text}'
    print(s)
    input_ids.append({'input_ids': tokenizer.encode(s, return_tensors='pt')[0]})

padded = tokenizer.pad(input_ids, padding='longest')
outputs = model.generate(**padded, max_length = 256)
b = tokenizer.batch_decode(outputs, skip_special_tokens=True)
for no, l in enumerate(tags):
    print(l, b[no])

Ekstrak person dari teks: olly saya nak secawan kopi dengan satu krim dan tiga gula , dan saya sekarang berada di penang airport
Ekstrak organisasi dari teks: olly saya nak secawan kopi dengan satu krim dan tiga gula , dan saya sekarang berada di penang airport
Ekstrak kopi dari teks: olly saya nak secawan kopi dengan satu krim dan tiga gula , dan saya sekarang berada di penang airport
Ekstrak nama parti politik dari teks: olly saya nak secawan kopi dengan satu krim dan tiga gula , dan saya sekarang berada di penang airport
Ekstrak lokasi dari teks: olly saya nak secawan kopi dengan satu krim dan tiga gula , dan saya sekarang berada di penang airport
Ekstrak syarikat dari teks: olly saya nak secawan kopi dengan satu krim dan tiga gula , dan saya sekarang berada di penang airport
Ekstrak masa dari teks: olly saya nak secawan kopi dengan satu krim dan tiga gula , dan saya sekarang berada di penang airport
Ekstrak quantity dari teks: olly saya nak secawan kopi dengan satu krim dan tiga gu

In [8]:
text = 'sya nak 1 teh o ais, dan saya sekarang berada di penang airport sambil minum starbucks'
tags = ['person', 'organisasi', 'kopi', 'nama parti politik', 'lokasi', 'syarikat', 'masa',
       'quantity', 'kuantiti', 'coffee', 'airport', 'drink', 'fnb', 'makanan dan minuman']
input_ids = []
for l in tags:
    s = f'Ekstrak {l} dari teks: {text}'
    print(s)
    input_ids.append({'input_ids': tokenizer.encode(s, return_tensors='pt')[0]})

padded = tokenizer.pad(input_ids, padding='longest')
outputs = model.generate(**padded, max_length = 256)
b = tokenizer.batch_decode(outputs, skip_special_tokens=True)
for no, l in enumerate(tags):
    print(l, b[no])

Ekstrak person dari teks: sya nak 1 teh o ais, dan saya sekarang berada di penang airport sambil minum starbucks
Ekstrak organisasi dari teks: sya nak 1 teh o ais, dan saya sekarang berada di penang airport sambil minum starbucks
Ekstrak kopi dari teks: sya nak 1 teh o ais, dan saya sekarang berada di penang airport sambil minum starbucks
Ekstrak nama parti politik dari teks: sya nak 1 teh o ais, dan saya sekarang berada di penang airport sambil minum starbucks
Ekstrak lokasi dari teks: sya nak 1 teh o ais, dan saya sekarang berada di penang airport sambil minum starbucks
Ekstrak syarikat dari teks: sya nak 1 teh o ais, dan saya sekarang berada di penang airport sambil minum starbucks
Ekstrak masa dari teks: sya nak 1 teh o ais, dan saya sekarang berada di penang airport sambil minum starbucks
Ekstrak quantity dari teks: sya nak 1 teh o ais, dan saya sekarang berada di penang airport sambil minum starbucks
Ekstrak kuantiti dari teks: sya nak 1 teh o ais, dan saya sekarang berada di pen

In [9]:
text = 'The Square is located by the historic Harvard Yard where you find buildings dating back as far as the early 18th century .'
tags = ['year', 'person', 'orang', 'org', 'people', 'tahun']
input_ids = []
for l in tags:
    s = f'Ekstrak {l} dari teks: {text}'
    print(s)
    input_ids.append({'input_ids': tokenizer.encode(s, return_tensors='pt')[0]})

padded = tokenizer.pad(input_ids, padding='longest')
outputs = model.generate(**padded, max_length = 256)
b = tokenizer.batch_decode(outputs, skip_special_tokens=True)
for no, l in enumerate(tags):
    print(l, b[no])

Ekstrak year dari teks: The Square is located by the historic Harvard Yard where you find buildings dating back as far as the early 18th century .
Ekstrak person dari teks: The Square is located by the historic Harvard Yard where you find buildings dating back as far as the early 18th century .
Ekstrak orang dari teks: The Square is located by the historic Harvard Yard where you find buildings dating back as far as the early 18th century .
Ekstrak org dari teks: The Square is located by the historic Harvard Yard where you find buildings dating back as far as the early 18th century .
Ekstrak people dari teks: The Square is located by the historic Harvard Yard where you find buildings dating back as far as the early 18th century .
Ekstrak tahun dari teks: The Square is located by the historic Harvard Yard where you find buildings dating back as far as the early 18th century .
year tiada
person tiada
orang tiada
org Harvard Yard
people tiada
tahun tiada


In [29]:
model.push_to_hub('finetune-zeroshot-ner-t5-small-standard-bahasa-cased', organization='mesolitica')

Cloning https://huggingface.co/mesolitica/finetune-zeroshot-ner-t5-small-standard-bahasa-cased into local empty directory.


Download file pytorch_model.bin:   0%|          | 8.74k/231M [00:00<?, ?B/s]

Download file spiece.model:   0%|          | 1.43k/784k [00:00<?, ?B/s]

Clean file spiece.model:   0%|          | 1.00k/784k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/231M [00:00<?, ?B/s]

Upload file pytorch_model.bin:   0%|          | 4.00k/231M [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/mesolitica/finetune-zeroshot-ner-t5-small-standard-bahasa-cased
   7fba56e..a4ee1b8  main -> main



'https://huggingface.co/mesolitica/finetune-zeroshot-ner-t5-small-standard-bahasa-cased/commit/a4ee1b893e38bcfec52fa940d41c83acb16daa43'

In [30]:
tokenizer.push_to_hub('finetune-zeroshot-ner-t5-small-standard-bahasa-cased', organization='mesolitica')

In [10]:
import re
import collections
import string

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
        return re.sub(regex, ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))


def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()


def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))


def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

In [11]:
import json
from tqdm import tqdm

In [12]:
f1, exact = [], []

with open('shuffled-test.json') as fopen:
    for l in tqdm(fopen):
        data = json.loads(l)['translation']
        input_ids = [{'input_ids': tokenizer.encode(data['prefix'] + data['src'], return_tensors='pt')[0]}]
        padded = tokenizer.pad(input_ids, padding='longest')
        outputs = model.generate(**padded, max_length = 256)
        a = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        b = data['tgt']
        f1.append(compute_f1(a, b))
        exact.append(compute_exact(a, b))

170139it [1:55:11, 24.62it/s]


In [15]:
import numpy as np

np.mean(f1), np.mean(exact)

(0.9512811620321475, 0.9208940924773273)